In [1]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report

---

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [2]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
file_path = Path("Resources/lending_data.csv")
df_credit_lending = pd.read_csv(file_path)

# Review the DataFrame
df_credit_lending.head()

loan_size  interest_rate  borrower_income  debt_to_income  num_of_accounts  \
0    10700.0          7.672            52800        0.431818                5   
1     8400.0          6.692            43600        0.311927                3   
2     9000.0          6.963            46100        0.349241                3   
3    10700.0          7.664            52700        0.430740                5   
4    10800.0          7.698            53000        0.433962                5   

   derogatory_marks  total_debt  loan_status  
0                 1       22800            0  
1                 0       13600            0  
2                 0       16100            0  
3                 1       22700            0  
4                 1       23000            0

### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [3]:
# Separate the data into labels and features

# Separate the y variable, the labels
y = df_credit_lending["loan_status"]

# Separate the X variable, the features
X = df_credit_lending.drop(columns="loan_status")

In [4]:
# Review the y variable Series
print(y)

0        0
1        0
2        0
3        0
4        0
        ..
77531    1
77532    1
77533    1
77534    1
77535    1
Name: loan_status, Length: 77536, dtype: int64


In [5]:
# Review the X variable DataFrame
X.head()


loan_size  interest_rate  borrower_income  debt_to_income  num_of_accounts  \
0    10700.0          7.672            52800        0.431818                5   
1     8400.0          6.692            43600        0.311927                3   
2     9000.0          6.963            46100        0.349241                3   
3    10700.0          7.664            52700        0.430740                5   
4    10800.0          7.698            53000        0.433962                5   

   derogatory_marks  total_debt  
0                 1       22800  
1                 0       13600  
2                 0       16100  
3                 1       22700  
4                 1       23000

### Step 3: Check the balance of the labels variable (`y`) by using the `value_counts` function.

In [6]:
# Check the balance of our target values
y.value_counts()

0    75036
1     2500
Name: loan_status, dtype: int64

### Step 4: Split the data into training and testing datasets by using `train_test_split`.

In [7]:
# Import the train_test_learn module
from sklearn.model_selection import train_test_split

# Split the data using train_test_split
# Assign a random_state of 1 to the function
X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=1,stratify=y)
X.head()


loan_size  interest_rate  borrower_income  debt_to_income  num_of_accounts  \
0    10700.0          7.672            52800        0.431818                5   
1     8400.0          6.692            43600        0.311927                3   
2     9000.0          6.963            46100        0.349241                3   
3    10700.0          7.664            52700        0.430740                5   
4    10800.0          7.698            53000        0.433962                5   

   derogatory_marks  total_debt  
0                 1       22800  
1                 0       13600  
2                 0       16100  
3                 1       22700  
4                 1       23000

---

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [8]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
allocated = LogisticRegression(solver='lbfgs', random_state=1)
# Fit the model using training data
logistic_model = allocated.fit(X_train, y_train)

### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [9]:
# Make a prediction using the testing data
predictions = logistic_model.predict(X_test)
pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)


Prediction  Actual
0               0       0
1               0       1
2               0       0
3               0       0
4               0       0
...           ...     ...
19379           0       0
19380           0       0
19381           0       0
19382           0       0
19383           0       0

[19384 rows x 2 columns]

### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [10]:
# Print the balanced_accuracy score of the model
balanced_accuracy_score(y_test, predictions)

0.9442676901753825

In [11]:
# Generate a confusion matrix for the model
from sklearn.metrics import confusion_matrix
testing_matrix = confusion_matrix(y_test, predictions)
print(testing_matrix)

[[18679    80]
 [   67   558]]


In [12]:
# Print the classification report for the model
target_names = ["Healthy Loan", "High-Risk Loan"]
testing_report = classification_report(y_test, predictions, target_names = target_names)
print(testing_report)

                precision    recall  f1-score   support

  Healthy Loan       1.00      1.00      1.00     18759
High-Risk Loan       0.87      0.89      0.88       625

      accuracy                           0.99     19384
     macro avg       0.94      0.94      0.94     19384
  weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question.

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** The model's 94% balanced accuracy score is highly reliable due to the lower number of high-risk loans. It correctly identifies 99.57% of healthy loans and 99.64% of actual healthy loans. For high-risk loans, it has an 87% prediction accuracy and identifies 89% of the actual high-risk loans. This accuracy, despite the scarcity of high-risk loans, should enable profitable lending by covering potential losses.

---

## Predict a Logistic Regression Model with Resampled Training Data

### Step 1: Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Be sure to confirm that the labels have an equal number of data points. 

In [13]:
# Import the RandomOverSampler module form imbalanced-learn
from imblearn.over_sampling import RandomOverSampler

# Instantiate the random oversampler model
# # Assign a random_state parameter of 1 to the model
random_oversample = RandomOverSampler(sampling_strategy='minority', random_state=1)

# Fit the original training data to the random_oversampler model
X_over, y_over = random_oversample.fit_resample(X_train, y_train)


In [14]:
# Count the distinct values of the resampled labels data
y_over.value_counts()
y_over
X_over

loan_size  interest_rate  borrower_income  debt_to_income  \
0          8500.0          6.722            43900        0.316629   
1         10000.0          7.360            49900        0.398798   
2          8400.0          6.708            43700        0.313501   
3         11000.0          7.779            53800        0.442379   
4          8900.0          6.924            45800        0.344978   
...           ...            ...              ...             ...   
112549    19000.0         11.217            86200        0.651972   
112550    18400.0         10.934            83500        0.640719   
112551    20700.0         11.904            92600        0.676026   
112552    18100.0         10.815            82400        0.635922   
112553    20400.0         11.785            91500        0.672131   

        num_of_accounts  derogatory_marks  total_debt  
0                     3                 0       13900  
1                     4                 0       19900  
2                     3                 0       13700  
3                     5                 1       23800  
4                     3                 0       15800  
...                 ...               ...         ...  
112549               12                 2       56200  
112550               12                 2       53500  
112551               14                 3       62600  
112552               11                 2       52400  
112553               13                 3       61500  

[112554 rows x 7 columns]

### Step 2: Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

In [15]:
# Instantiate the Logistic Regression model
allocated = LogisticRegression(solver='lbfgs', random_state=1)

# Fit the model using the resampled training data
logistic_model = allocated.fit(X_over, y_over)


# Make a prediction using the testing data
predictions = logistic_model.predict(X_test)
pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)


Prediction  Actual
0               0       0
1               1       1
2               0       0
3               0       0
4               0       0
...           ...     ...
19379           0       0
19380           0       0
19381           0       0
19382           0       0
19383           0       0

[19384 rows x 2 columns]

### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [16]:
# Print the balanced_accuracy score of the model 
balanced_accuracy_score(y_test, predictions)

0.9959744975744975

In [17]:
# Generate a confusion matrix for the model
testing_matrix= confusion_matrix(y_test, predictions)
print(testing_matrix)

[[18668    91]
 [    2   623]]


In [18]:
# Print the classification report for the model
target_names = ["Healthy Loan", "High-Risk Loan"]
testing_report = classification_report(y_test, predictions, target_names=target_names)
print(testing_report)

                precision    recall  f1-score   support

  Healthy Loan       1.00      1.00      1.00     18759
High-Risk Loan       0.87      1.00      0.93       625

      accuracy                           1.00     19384
     macro avg       0.94      1.00      0.96     19384
  weighted avg       1.00      1.00      1.00     19384



### Step 4: Answer the following question

**Question:** How well does the logistic regression model, fit with oversampled data, predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** The balanced accuracy score increased by 5.5% to 99.59%, with almost 100% precision for healthy loans (99.51% manually calculated) and 87% precision for high-risk loans. The recall for high-risk loans improved significantly, rising from 89% to nearly 100%. 